![](https://howtocure.com/wp-content/uploads/2019/10/Cassava-Leaves.jpg)

Cassava originated in South America and was introduced to Africa in relatively recent times. It is known to be a very drought-tolerant crop with the ability to yield even when planted in poor soils. When cassava was first grown in Africa, it was used for subsidiary purposes though it is now considered to be one of the most important food staple crops on the continent. Its production is moving toward an industrialized system in which plant material is used for a variety of products including starch, flour, and animal feed.

As the second-largest provider of carbohydrates in Africa, cassava is a key food security crop grown by smallholder farmers because it can withstand harsh conditions. At least 80% of household farms in Sub-Saharan Africa grow this starchy root, but viral diseases are major sources of poor yields. With the help of data science, it may be possible to identify common diseases so they can be treated.

Existing methods of disease detection require farmers to solicit the help of government-funded agricultural experts to visually inspect and diagnose the plants. This suffers from being labor-intensive, low-supply and costly. As an added challenge, effective solutions for farmers must perform well under significant constraints, since African farmers may only have access to mobile-quality cameras with low-bandwidth.

![](https://www.bing.com/images/search?view=detailV2&ccid=LnFdsek3&id=304C6B3F770790F5BC0604B8AB370F975A7AE3B8&thid=OIP.LnFdsek3pT7Ed_vrgBv2kQHaE6&mediaurl=http%3a%2f%2fgondwanaecotours.com%2fwp-content%2fuploads%2f2016%2f01%2fcassava-1280x850.jpg&exph=850&expw=1280&q=cassava+leaf&simid=608055820647793674&ck=C7F803AB5B29A6DFAD14B6F61A022DB5&selectedIndex=2&FORM=IRPRST&ajaxhist=0)

# **IMPORT RELEVANT LIBRARIES**

In [ ]:






import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.optimizers import Adam

# ignoring warnings
import warnings
warnings.simplefilter("ignore")

import os, cv2, json
from PIL import Image

1. ** PROVIDING PATH FOR THE DIRECTORY**

In [ ]:
import os 


WORK_DIR = '../input/cassava-leaf-disease-classification'
os.listdir(WORK_DIR)

# **BASIC LOOK INTO THE DATA**

In [ ]:
print('Train images: %d' %len(os.listdir(
    os.path.join(WORK_DIR, "train_images"))))

In [ ]:
with open(os.path.join(WORK_DIR, "label_num_to_disease_map.json")) as file:
    print(json.dumps(json.loads(file.read()), indent=4))

In [ ]:
train_labels = pd.read_csv(os.path.join(WORK_DIR, "train.csv"))
train_labels.head()

# **STAGES OF BUILDING A MODEL**

PARAMETERS

In [ ]:
# target size and number of epochs can be tuned to your needs for obtaining better accuracy



BATCH_SIZE = 8
STEPS_PER_EPOCH = len(train_labels)*0.8 / BATCH_SIZE
VALIDATION_STEPS = len(train_labels)*0.2 / BATCH_SIZE
EPOCHS = 20
TARGET_SIZE = 350

IMAGEDATAGENERATOR

In [ ]:





train_labels.label = train_labels.label.astype('str')

train_datagen = ImageDataGenerator(validation_split = 0.2,
                                     preprocessing_function = None,
                                     rotation_range = 45,
                                     zoom_range = 0.2,
                                     horizontal_flip = True,
                                     vertical_flip = True,
                                     fill_mode = 'nearest',
                                     shear_range = 0.1,
                                     height_shift_range = 0.1,
                                     width_shift_range = 0.1)


train_generator = train_datagen.flow_from_dataframe(train_labels,
                         directory = os.path.join(WORK_DIR, "train_images"),
                         subset = "training",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,
                         class_mode = "sparse")


validation_datagen = ImageDataGenerator(validation_split = 0.2)

validation_generator = validation_datagen.flow_from_dataframe(train_labels,
                         directory = os.path.join(WORK_DIR, "train_images"),
                         subset = "validation",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,
                         class_mode = "sparse")

MODEL

SELECT YOUR MODEL ACCORDINGLY



important thing to remmember : chose a model with lesser parameters and higher accuracy
#here i have taken effeciantnetB0

![](https://miro.medium.com/max/828/0*09AED_CjE-PUFxKC.png)

In [ ]:
def create_model():
    conv_base = EfficientNetB3(include_top = False, weights = None,
                               input_shape = (TARGET_SIZE, TARGET_SIZE, 3))
    model = conv_base.output
    model = layers.GlobalAveragePooling2D()(model)
    model = layers.Dense(5, activation = "softmax")(model)
    model = models.Model(conv_base.input, model)

    model.compile(optimizer = Adam(lr = 0.001),
                  loss = "sparse_categorical_crossentropy",
                  metrics = ["acc"])
    return model

SUMMARY

In [ ]:
model = create_model()
model.summary()

In [ ]:
print('Our EfficientNet CNN has %d layers' %len(model.layers))

In [ ]:
model.load_weights('../input/cassava-leaf-disease-models/basic_EfNetB0_imagenet_512.h5')

TRAINING

In [ ]:
model_save = ModelCheckpoint('./EffNetB0_512_8_best_weights.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, 
                              patience = 2, min_delta = 0.001, 
                              mode = 'min', verbose = 1)


history = model.fit(
    train_generator,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = EPOCHS,
    validation_data = validation_generator,
    validation_steps = VALIDATION_STEPS,
    callbacks = [model_save, early_stop, reduce_lr]
)

SAVE THE MODEL

In [ ]:
model.save('./EffNetB0_512_8.h5')

# PREDICTION


In [ ]:
submission = pd.read_csv(os.path.join(WORK_DIR, "sample_submission.csv"))
submission

In [ ]:
preds = []

for image_id in submission.image_id:
    image = Image.open(os.path.join(WORK_DIR,  "test_images", image_id))
    image = image.resize((TARGET_SIZE, TARGET_SIZE))
    image = np.expand_dims(image, axis = 0)
    preds.append(np.argmax(model.predict(image)))

submission['label'] = preds
submission

In [ ]:
submission.to_csv('submission.csv', index = False)

If you like the notebook give me a upvote 

![](https://designshop-6aa0.kxcdn.com/photos/have-a-happy-day-greeting-card-balloon-quote-postcard-2411_21.jpg)